In [111]:
import joblib
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [74]:
strings_list = [
    "@KeepinUpWKris I am so jealous, hope you had a great time in vegas! how did you like the ACM's?! LOVE YOUR SHOW!! ",
    "@LovesBrooklyn2 he has that effect on everyone",
    "@westy154 Me too   I don't do suits.  I look like I'm in fancy dress.  Especially with shoulder pads.",
    "dang! i'm lazy  i've begun three short stories in the last three weeks... and never finished anything. will. have. more. focus.",
    "@yayfuckbuddies haha I would but you won't"
]

# Create a DataFrame with one column named 'Text'
data = pd.DataFrame({'TweetText': strings_list})

In [75]:
stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    clean_text=' '.join([word for word in text.split() if word not in stopwords])
    return clean_text
# Removing Twitter Handles (@user)
data['Clean_TweetText'] = data['TweetText'].str.replace("@", "") 
# Removing links
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace(r"http\S+", "") 
# Removing Punctuations, Numbers, and Special Characters
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace("[^a-zA-Z]", " ")
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda text : remove_stopwords(text.lower()))
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: nltk.word_tokenize(x))
# Now let’s stitch these tokens back together
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x]))
# Removing small words
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data.head()

C:\Users\lohit.ashwa\AppData\Local\Temp\ipykernel_3060\2909059753.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Clean_TweetText'] = data['Clean_TweetText'].str.replace(r"http\S+", "")
C:\Users\lohit.ashwa\AppData\Local\Temp\ipykernel_3060\2909059753.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Clean_TweetText'] = data['Clean_TweetText'].str.replace("[^a-zA-Z]", " ")


,TweetText,Clean_TweetText
0,"@KeepinUpWKris I am so jealous, hope you had a...",keepinupwkris jealous hope great time vegas li...
1,@LovesBrooklyn2 he has that effect on everyone,lovesbrooklyn effect everyone
2,@westy154 Me too I don't do suits. I look l...,westy suits look like fancy dress especially s...
3,dang! i'm lazy i've begun three short stories...,dang lazy begun three short stories last three...
4,@yayfuckbuddies haha I would but you won't,yayfuckbuddies haha would


In [85]:
data2=pd.read_csv(r'C:\Users\lohit.ashwa\Downloads\archive\processed data.csv',usecols=["TweetText","Clean_TweetText"])
data2

,TweetText,Clean_TweetText
0,I LOVE @Health4UandPets u guys r the best!!,love health uandpets guys best
1,im meeting up with one of my besties tonight! ...,meeting besties tonight cant wait girl talk
2,"@DaRealSunisaKim Thanks for the Twitter add, S...",darealsunisakim thanks twitter sunisa meet sho...
3,Being sick can be really cheap when it hurts t...,sick really cheap hurts much real food plus fr...
4,@LovesBrooklyn2 he has that effect on everyone,lovesbrooklyn effect everyone
...,...,...
19995,stupid dvds stuffing up the good bits in jaws.,stupid dvds stuffing good bits jaws
19996,@Dandy_Sephy No. Only close friends and family...,dandy sephy close friends family afraid work c...
19997,CRAP! After looking when I last tweeted... WHY...,crap looking last tweeted early like
19998,Its Another Rainboot day,another rainboot


In [86]:
data = pd.concat([data,data2],axis = 0)

In [87]:
data

,TweetText,Clean_TweetText
0,"@KeepinUpWKris I am so jealous, hope you had a...",keepinupwkris jealous hope great time vegas li...
1,@LovesBrooklyn2 he has that effect on everyone,lovesbrooklyn effect everyone
2,@westy154 Me too I don't do suits. I look l...,westy suits look like fancy dress especially s...
3,dang! i'm lazy i've begun three short stories...,dang lazy begun three short stories last three...
4,@yayfuckbuddies haha I would but you won't,yayfuckbuddies haha would
...,...,...
19995,stupid dvds stuffing up the good bits in jaws.,stupid dvds stuffing good bits jaws
19996,@Dandy_Sephy No. Only close friends and family...,dandy sephy close friends family afraid work c...
19997,CRAP! After looking when I last tweeted... WHY...,crap looking last tweeted early like
19998,Its Another Rainboot day,another rainboot


In [90]:
# Option 1: Drop rows with NaN values
data = data.dropna(subset=['Clean_TweetText'])


In [91]:
count_vectorizer = CountVectorizer(stop_words='english') 
cv = count_vectorizer.fit_transform(data['Clean_TweetText'])
cv.shape

(19955, 24904)

In [104]:
cv[:5]

<5x24904 sparse matrix of type '<class 'numpy.int64'>'
	with 29 stored elements in Compressed Sparse Row format>

In [98]:
model2=joblib.load(r'C:\Users\lohit.ashwa\Desktop\New folder\xgbc_model.pkl')

In [105]:
prediction=model2.predict(cv[:5])

In [106]:
prediction

array([0, 1, 1, 0, 1])

In [110]:
counts = Counter(prediction)
if(counts[1]== counts[0]):
    print("Neutral")
elif(counts[1]> counts[0]):
    print("Positive")
else:
    print("Negative")


Positive
